In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project'

In [5]:
import os

def list_folder_structure(root_dir):
    for root, dirs, files in os.walk(root_dir):
        level = root.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# Adjust this path to the actual location of the Ultralytics package on your system
# For example, it might be something like below if installed in an Anaconda environment
package_path = 'c:/Users/vishw/anaconda3/envs/capstone/Lib/site-packages/ultralytics'

print(f"Listing folder structure for: {package_path}")
list_folder_structure(package_path)


Listing folder structure for: c:/Users/vishw/anaconda3/envs/capstone/Lib/site-packages/ultralytics
ultralytics/
    detect.py
    train.py
    val.py
    __init__.py
    assets/
        bus.jpg
        zidane.jpg
    cfg/
        default.yaml
        __init__.py
        datasets/
            Argoverse.yaml
            carparts-seg.yaml
            coco-pose.yaml
            coco.yaml
            coco128-seg.yaml
            coco128.yaml
            coco8-pose.yaml
            coco8-seg.yaml
            coco8.yaml
            crack-seg.yaml
            dota8.yaml
            DOTAv1.5.yaml
            DOTAv1.yaml
            GlobalWheat2020.yaml
            ImageNet.yaml
            missile_data.yaml
            Objects365.yaml
            open-images-v7.yaml
            package-seg.yaml
            SKU-110K.yaml
            tiger-pose.yaml
            VisDrone.yaml
            VOC.yaml
            xView.yaml
        models/
            rt-detr/
                rtdetr-l.yaml
            

In [6]:
from dataclasses import dataclass

@dataclass
class ModelTrainConfig:
    output_dir: str
    runs_dir: str
    training_data_yaml: str
    epochs: 3


In [7]:
from YOLOv8_objectdetection.constants.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from YOLOv8_objectdetection.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_model_train_config(self) -> ModelTrainConfig:
        config = self.config.model_training

        model_train_config = ModelTrainConfig(
            output_dir=config.output_dir,
            runs_dir=config.runs_dir,
            training_data_yaml= config.training_data_yaml,
            epochs=config.epochs
        )

        return model_train_config

In [9]:
import os
from shutil import copy2
from ultralytics import YOLO
from pathlib import Path

In [16]:
class ModelTraining:
    def __init__(self, config: ModelTrainConfig):
        self.data_yaml_path = config.training_data_yaml
        self.epochs = config.epochs
        self.output_dir = Path(config.output_dir).resolve()
        self.base_run_dir = Path(config.runs_dir).resolve()

    def train_and_save_model(self):
        # Load the pretrained model
        model = YOLO("yolov8n.pt")  # Consider making this dynamic if needed

        # Train the model with the custom dataset
        print(f"Training with {self.data_yaml_path} for {self.epochs} epochs...")
        model.train(data=self.data_yaml_path, epochs=self.epochs)

        # Assuming the best weights are saved by default
        # Create the output directory if it doesn't exist
        os.makedirs(self.output_dir, exist_ok=True)

        # Dynamically find the most recent training run directory within 'runs/detect'
        try:
            print(f"Looking in directory: {self.base_run_dir}")  # Diagnostic print
            detect_runs = [run for run in self.base_run_dir.iterdir() if run.is_dir() and run.name.startswith('train')]
            print(f"Found detect training runs: {detect_runs}")  # Diagnostic print
            last_run_path = sorted(detect_runs, key=os.path.getmtime, reverse=True)[0] / 'weights'
            print(f"Most recent training run: {last_run_path}")  # Diagnostic print
        except IndexError:
            raise FileNotFoundError(f"No training runs found in {self.base_run_dir}. Please ensure training has completed successfully.")

        source_best_weight_path = last_run_path / 'best.pt'

        # Check if best.pt already exists in the output directory
        target_best_weight_path = self.output_dir / 'best.pt'
        if os.path.exists(target_best_weight_path):
            print(f"'best.pt' already exists in the output directory and will be overwritten.")

        # Copy the best model weights to the specified output directory
        copy2(source_best_weight_path, target_best_weight_path)
        print(f"Best model weights saved to {target_best_weight_path}")


In [17]:
if __name__ == "__main__":
    config_manager = ConfigurationManager()  # Assuming default paths are set within the class
    model_train_config = config_manager.get_model_train_config()

    trainer = ModelTraining(model_train_config)
    trainer.train_and_save_model()

[2024-02-25 21:00:12,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-25 21:00:12,172: INFO: common: yaml file: params.yaml loaded successfully]


Training with missile_data.yaml for 3 epochs...
Ultralytics YOLOv8.1.18 🚀 Python-3.12.1 torch-2.2.1+cpu CPU (AMD Ryzen 5 7640HS w/ Radeon 760M Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=missile_data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

train: Scanning C:\Users\vishw\Documents\cap_project_tulasi\YOLO-missile-detection-capstone-project\missile_data\labels\train.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]
val: Scanning C:\Users\vishw\Documents\cap_project_tulasi\YOLO-missile-detection-capstone-project\missile_data\labels\val.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]

Plotting labels to runs\detect\train2\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
MLflow: logging run_id(d4b08b273ba44a10a99df4f73c76b7d3) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        3/3         0G      1.844      2.893      1.634         39        640: 100%|██████████| 3/3 [00:09<00:00,  3.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

                   all         12         24    0.00583      0.875       0.39     0.0955



3 epochs completed in 0.010 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.18 🚀 Python-3.12.1 torch-2.2.1+cpu CPU (AMD Ryzen 5 7640HS w/ Radeon 760M Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


                   all         12         24    0.00556      0.833      0.349      0.105
Speed: 8.6ms preprocess, 61.3ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs\detect\train2
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'
Looking in directory: C:\Users\vishw\Documents\cap_project_tulasi\YOLO-missile-detection-capstone-project\runs\detect
Found detect training runs: [WindowsPath('C:/Users/vishw/Documents/cap_project_tulasi/YOLO-missile-detection-capstone-project/runs/detect/train'), WindowsPath('C:/Users/vishw/Documents/cap_project_tulasi/YOLO-missile-detection-capstone-project/runs/detect/train2')]
Most recent training run: C:\Users\vishw\Documents\cap_project_tulasi\YOLO-missile-detection-capstone-project\runs\detect\train2\weights
'best.pt' already exists in the output directory and will be overwritten.
Best model weights saved to C:\Users\vishw\Documents\cap_project_tulasi\YOLO-missile-detection-capstone-proje